# Tesla Stock Price Prediction - Simple RNN Model

## Objective
- Build SimpleRNN models for 1-day, 5-day, and 10-day predictions
- Use EarlyStopping and ModelCheckpoint
- Evaluate using MSE

### Fix TensorFlow not found
If you get `ModuleNotFoundError: No module named 'tensorflow.python'`, run the cell below **once** so TensorFlow is installed for this notebook's Python (the kernel). Then restart the kernel and run all cells again.

In [ ]:
# Run this cell once if TensorFlow fails to import, then restart kernel (Kernel → Restart) and run all again.
!pip install "tensorflow>=2.15.0,<2.17.0"

In [ ]:
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error

PROJECT_ROOT = Path('..').resolve()
DATA_PATH = PROJECT_ROOT / 'data' / 'processed'
MODELS_PATH = PROJECT_ROOT / 'models'
MODELS_PATH.mkdir(exist_ok=True)

In [ ]:
def build_simple_rnn(lookback=60, units=50, dropout=0.2):
    model = Sequential([
        SimpleRNN(units, return_sequences=False, input_shape=(lookback, 1)),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
horizons = [1, 5, 10]
results = []

for horizon in horizons:
    X_train = np.load(DATA_PATH / f'X_train_{horizon}d.npy')
    y_train = np.load(DATA_PATH / f'y_train_{horizon}d.npy')
    X_test = np.load(DATA_PATH / f'X_test_{horizon}d.npy')
    y_test = np.load(DATA_PATH / f'y_test_{horizon}d.npy')
    
    model = build_simple_rnn(lookback=60, units=50, dropout=0.2)
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ModelCheckpoint(MODELS_PATH / f'simple_rnn_{horizon}day.h5', save_best_only=True, monitor='val_loss')
    ]
    
    history = model.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=32, callbacks=callbacks, verbose=1)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results.append({'horizon': horizon, 'mse': mse})
    print(f"{horizon}-day SimpleRNN MSE: {mse:.6f}")

print("\nResults:", results)